# Char-based text generation with LSTM

In [2]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
df_texts = pd.read_csv("/Users/yuvlo/Spot-the-bot/Russian/RuPreprocessedWithSep.csv")

In [4]:
text = ""
for i in tqdm(range(df_texts.shape[0])):
    text += df_texts.loc[i, 'preprocessed_text']
    text += ' '

100%|███████████████████████████████████| 10403/10403 [00:03<00:00, 2934.05it/s]


In [5]:
len(text.split())

127390630

In [6]:
len(set(text.split()))

106380

In [7]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)
 
    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char
 
sequence, char_to_idx, idx_to_char = text_to_seq(text)

[' ', 'о', 'а', 'т', 'е', 'с', 'ь', 'н', 'и', 'р', 'в', 'л', 'к', 'п', 'д', 'й', 'ы', 'я', 'м', 'у', '.', 'з', 'б', 'ч', 'г', 'ж', 'ш', 'х', 'ц', 'щ', 'э', 'ю', 'ф', 'ъ', '-']


In [5]:
len(char_to_idx)

35

In [9]:
SEQ_LEN = 1024
BATCH_SIZE = 128

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [10]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=256, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 10000
loss_avg = []

In [12]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [ ]:
for epoch in tqdm(range(n_epochs)):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

  0%|▏                                     | 49/10000 [00:25<1:21:39,  2.03it/s]

Loss: 2.473687274456024


  0%|▏                                     | 50/10000 [00:26<1:37:15,  1.71it/s]

 стание . потодить посторо . подолод . присоветь . моро сободить . . смодовать . просторовать сатать . . ставать ставать . просто прододать ставать поделенный . прапрать . . подразать . дело . просто п


  1%|▎                                    | 100/10000 [00:51<1:26:28,  1.91it/s]

...

 20%|███████▏                            | 2000/10000 [17:00<1:20:19,  1.66it/s]

Loss: 1.2867756748199464
 . сообщать . подвигаться . подавать . просто весь пока понимать выходить весь получать показываться . просто подруга . старый подруга . получать стол . получать . подобный . подобный подобный приходит


 20%|███████▍                            | 2050/10000 [17:27<1:12:24,  1.83it/s]

Loss: 1.2831489062309265
 . показываться служба поставлять . получаться . простой весь подозреваться сказать простой просто привычный случай обращаться попросить приходиться показывать . подобный собираться получаться подрезат


 21%|███████▍                            | 2080/10000 [17:42<1:04:48,  2.04it/s]

In [ ]:
torch.save(model, f="/Users/yuvlo/Spot-the-bot/TextGenBot/lstm_bot.pth")

In [13]:
model = torch.load("/Users/yuvlo/Spot-the-bot/TextGenBot/lstm_bot.pth")

In [14]:
model

TextRNN(
  (encoder): Embedding(35, 128)
  (lstm): LSTM(128, 256, num_layers=2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=35, bias=True)
)

In [24]:
def generate_text(df_texts: pd.DataFrame,
                  split_range: int = 100,
                  start_length: int = 10,
                  prediction_length: int = 750) -> pd.DataFrame:
    df_bot = pd.DataFrame(columns=['predicted_text'])
    for i in tqdm(range(df_texts.shape[0])):
        text = df_texts.loc[i, 'preprocessed_text']
        splitted_text = text.split()
        predicted_text = ""
        for j in range(0, len(splitted_text), split_range):
            fragment = splitted_text[j:j+split_range]
            start_text = ' '.join(fragment[:start_length])
            predicted_fragment = evaluate(model,
                                          char_to_idx,
                                          idx_to_char,
                                          temp=0.3,
                                          prediction_len=prediction_length,
                                          start_text=start_text + ' ')
            predicted_fragment = ' '.join(predicted_fragment.split()[start_length:-1])
            print(f"start: {start_text}, predict: {predicted_fragment, len(predicted_fragment.split())}")
            predicted_text += predicted_fragment
            predicted_text += ' '
        df_bot.loc[i, 'predicted_text'] = predicted_text
    return df_bot

In [41]:
def generate_text(df_texts: pd.DataFrame,
                  split_range: int = 100) -> pd.DataFrame:
    df_bot = pd.DataFrame(columns=['predicted_text'])
    prediction_length = round(split_range * 7.5)
    for i in tqdm(range(df_texts.shape[0])):
        text = df_texts.loc[i, 'preprocessed_text']
        splitted_text = text.split()
        predicted_text = ""
        for j in range(0, len(splitted_text), split_range):
            fragment = splitted_text[j:j+split_range]
            start_text = ' '.join(fragment)
            predicted_fragment = evaluate(model,
                                          char_to_idx,
                                          idx_to_char,
                                          temp=0.3,
                                          prediction_len=prediction_length,
                                          start_text=start_text + ' ')
            predicted_fragment = ' '.join(predicted_fragment.split()[split_range:-1])
            print(f"start: {start_text} \n, predict: {predicted_fragment, len(predicted_fragment.split())}")
            predicted_text += predicted_fragment
            predicted_text += ' '
        df_bot.loc[i, 'predicted_text'] = predicted_text
    return df_bot

In [51]:
generate_text(df_texts[2110:2111].reset_index(drop=True))

  0%|                                                     | 0/1 [00:00<?, ?it/s]

start: серебряный ангел . серебряный ангел джоанна линдсей серебряный ангел роман перевод . издание русский язык глава год улица торговец жемчуг закрывать свой лавка дожидаться пение муэдзин призывать правоверный молитва . намаз мало десять минута становиться старый костя болеть пригибать земля поэтому каждый день выходить рано . пока сила оставлять ходить молиться мечеть . намерен расстилать молитвенный коврик лавка подобно менее благочестивый сосед . поскольку торговец жемчуг оказываться час единственный человек улица становиться единственный свидетель происходить глаз убийство . молодой турок бежать здоровенный мужчина черный пронестись нос обращать малый внимание . немного свертывать угол скрываться вид торговец жемчуг приходиться наблюдать кошмар 
, predict: ('мочь получать . слово стоять дом который получать положение старик приходить подавать подвигаться . заканчивать столько положение . весь таки просто понимать . становиться становиться высокий привычка . подумать подниматься 

start: след покойник . множество способ избавляться . привыкать иметь значение . хотеть дворец знать курьер удаваться выполнять поручение . дей узнавать находить труп . это пустой трата время хотеть знать мнение выпаливать стараться скрывать свой раздражение . пожалуй рискнуть сделать свой работа прямо дворец . знать . мочь повезти суметь зарабатывать самый большой кошелек который причитаться голова грек засмеяться . брат обменяться красноречивый взгляд . оба сомневаться вряд вновь увидеть живой попытаться проникать дворец . четыре покушение жизнь дея охранять усердный либо рано . любой попытаться убивать подписывать смертный приговор . казнить неудачник подвергать пытка начинать называть имя . имя 
, predict: ('странный . просто скрывать . последний слово подобный стол . последний стол . просто просто получать . подробность . подобный старый след подписывать . становиться . старый сторона . получать положение сказать . весь таки усталость . просто приходиться высокий стол . направлять

start: получать твой письмо . твой племянница траур заканчиваться должно становиться выезжать . надеяться написать это подробно . наверное некогда писать готовить выход свет . это понимать мочь . чудесный девушка . должно весь окрестный жених бегать . достойный . хотя дорогой это иметь значение . выбирать приезжать . ждать дождаться новый встреча любезный . знать муж дочь опускать письмо колено потирать глаз . скучно читать письмо недоумевать женщина умудряться исписывать десять двенадцать страница полный чепуха . каждый . подумать год совместный учеба школа каждый несколько месяц письмо полный нелепый сплетня . приходиться читать знать написать полезный . пробежать глаз несколько 
, predict: ('свой половина . подниматься возможность . весь старик . получать . просто сказать . становиться . пойти просто подумать . становиться свой проводить воздух подавать подвигаться полный сказать . находить свой хотеть подходить . стоять . просто просто сказать . становиться становиться подымать сто

start: . называть маленький хрупкий слишком высокий пять половина фут рост весьма привлекательный возможно излишне обходить мужской внимание будущее . выбирать . смочь находить хороший муж опекун вообще иметь шанс счастие вздыхать . человек отвечать письмо близкий время приходиться серьезно подумать самый отвозить . девушка должный появляться общество соответствие свой положение состояние . отказывать это похоже оно приобретать опасный враг лицо . достаточно влиятельный друг доставлять масса неприятность американский кузен который выполнять свой обязанность . тетя вернуться раздаваться кухня голос мгновение входить гостиная . выбирать отличный кусок мясо обед почка завтрак . миссис смит просить передавать девушка закатить глаз далеко посылать 
, predict: ('. весь странный странный приходить просто каждый сторона стоять старый полный принимать стол . некоторый положение . отвечать . получать свой весь сторона высокий странный странный полоса . почему . пора последний половина . поставля

...

start: таки приходиться родной дядя . отвечать несколько удивленный . соглашаться план непременно увидеть малютка весь жена ребенок . сказать . понимать ответ вопрос который хотеть задавать мягкий взгляд недоумение брат рассердиться . сын верблюжий дерьмо . сказать позвать сюда . оставлять это улыбаться . честно говорить это доходить . приход обсуждать это проблема разведение лошадь . разведение лошадь . дело привозить отличный пара чистокровный скакун . гнев лицо уступать место выражение неподдельный детский восхищение . привозить сказать весело . хотеться узнавать иметь вид говорить причина приезд поежиться . вообще это идея сказать оправдываться . сначала отказываться обсуждать . изо день 
, predict: ('. признаваться подымать полный стол случай постоянно поговорить . получать . равный поставлять просто понимать . забывать . подумать . последний стол . становиться просто построить предмет стоять становиться состоять странно постель . получать . просто . признавать . пропадать . подоз

start: раб подкупать дюжина шпионить сообщать весь перемещение изменять внешность мочь выходить уверенность это заранее становиться известно убийца . делать далеко разбираться дворец иметь лишь выход существенно облегчать наблюдение кивнуть . время время наблюдатель терять терпение посылать наемник прямо сюда надеяться удаваться заставать врасплох убивать . далее прошлый месяц удаваться проникать спальня . убивать стоящий дверь стражник попытаться подползать кровать который спать . счастие телохранитель оказываться бдительный успевать сделать собака решето успевать укусить . серебряный ангел конец ознакомительный фрагмент . текст предоставлять . прочитывать книга целиком купить полный легальный версия . безопасно оплачивать книга банковский карта счет мобильный телефон платежный терминал 
, predict: ('просто поступать приставать столько королевский голос . помнить становиться подробность . слушать . принимать старый большой совершенно предполагать надежда . весь сторона . образ подобны

100%|█████████████████████████████████████████████| 1/1 [01:23<00:00, 83.97s/it]

start: салон бонусный карта удобный способ . серебряный ангел примечание шебека тип парусный судно . переть вернуться 
, predict: ('старый просто сказать . собираться постоянно поверять . подразделение . просто спрашивать . получать . подавать . принимать стол . становиться', 21)


,predicted_text
0,мочь получать . слово стоять дом который получ...


In [50]:
df_texts[2110:2111].reset_index(drop=True)

,preprocessed_text
0,серебряный ангел . серебряный ангел джоанна ли...


In [43]:
text = df_texts.loc[2110, 'preprocessed_text']

In [32]:
splitted_text = text.split()
predicted_text = ""
for j in range(0, len(splitted_text), 100):
    fragment = splitted_text[j:j+100]
    print(j, j+100, fragment)

0 100 ['надеяться', 'повезти', 'прохрипеть', 'судорожно', 'сжинать', 'рука', 'умирать', 'оставлять', 'ладонь', 'крохотный', 'угловатый', 'предмет', '.', 'становиться', 'смотреть', 'это', 'просто', 'плотно', 'сжинать', 'палец', '.', 'заостряться', 'лицо', 'медленно', 'утекать', 'последний', 'капля', 'жизнь', 'превращать', 'лицо', 'застывший', 'мертвый', 'маска', '.', 'прощать', 'старик', '.', 'хотя', 'старик', 'пятьдесят', '.', 'покоиться', 'мир', 'старый', 'пропойца', '.', 'неплохой', 'человек', 'прогудеть', 'стоять', 'спина', 'толстый', 'увалень', 'ответственный', 'наш', 'жилой', 'блок', 'пара', 'минута', 'тело', 'прийти', '.', 'понимать', '.', 'обида', 'ладно', '.', 'никакой', 'обида', 'оборачиваться', 'отвечать', 'никакой', 'обида', '.', 'весь', 'свой', 'вещь', 'завещать', 'свидетель', '.', 'записывать', 'зафиксировать', '.', 'забирать', 'хотеть', '.', 'матрас', 'оставлять', 'баланс', 'числиться', '.', 'подушка', '.', 'успевать', 'заканчивать', 'фраза', 'дверь', 'отсек', 'завывать',

In [25]:
text = generate_text(df_texts[:1], )

  0%|                                                     | 0/1 [00:00<?, ?it/s]

start: надеяться повезти прохрипеть судорожно сжинать рука умирать оставлять ладонь крохотный, predict: ('старый дело . следовать поступать . замечать . случай последний страх . подумать поставлять половина . помещик . открывать стол . сказать . подумать весь специальный . весь получать сердце . старик последний долго который весь сторона . подниматься . пора состояние становиться продолжать просто следовать поставлять полный просто просто подумать представлять мочь подходить рука столько противный просто просто сказать противоположный установка подвергаться положение . просто подобный старый принимать становиться . просто просто поставлять проговаривать подобный образ просто спрашивать . подымать подобный весь подобный судьба . становиться становиться просто постоянно подходить подниматься . подобный подобный . подобный просто видеть', 99)
start: сервомотор уезжать вверх комната вваливаться трое смутно знакомый личность ., predict: ('просто сказать сторона . полезный постель . приходи

start: граница обитаемый пространство . колонизация планета разработка астероидный поле разведывание, predict: ('весь стройный стол . свой приветствовать . полагать голова . получать . постель . большой безопасность подходить . последний растерянный волос положение . странный который слово . просто просто подниматься . подобный день начинать приходить представлять свой улыбка . подобный слово . принимать стол . посмотреть положение . просто позволять . позволять подобный комната . подозревать . старик . подходить приставать . весь просто слово поставлять положение поступать . просто понимать стоять обращаться подниматься помогать . открывать король . стоять весь сторона просто подниматься половина последний половина . весь старик становиться пока пристально приходить весь подруга старый волос . направляться забывать . подобный мало восклицать .', 106)
start: пшик . станция оказываться практически тупик . некогда мощный поток, predict: ('оставаться достаточно . просто подымать свой случ

start: транзит предназначать работа столь огромный суд . это образно говорить, predict: ('подумать . подходить скрывать голова полагать просто последний страшный просто сказать . посмотреть . стоять столик . подавать старый сторона подумать . весь положение подавать просить приказывать собираться . получать рука . случаться . просто подавать день подводить направляться сторона . получать . приходить политика . подобный постоянно последний странный полный время продолжать подумать . поступать . понимать постоянно стоять дело . становиться спрашивать . стоять подарок . необходимо подобный стол . становиться совершенно подавать . напротив полный день . подходить . почему просто подумать полагать свой большой комната становиться спрашивать . это замечать . проводить отвечать . возможно подумать . просто постоянно построить старый', 105)
start: отец бесценок подарок пятнадцатилетие . поздно робот оставаться лишь кусок, predict: ('странный день помогать . сказать . подобный . просто подобный

start: кивнуть перепрыгивать ступенька помчаться свой комната . проводить карточка замок, predict: ('получать . получать молодой стол подавать возможность . старик получать возможность . пока подражать . дело приходить . просто находить . случай стоять приказывать . просто поставлять конец . просто подымать неприятный полный просто противник подымать просто сделать . полагать . сказать . подобный старик просто оставаться слово получать старший просто закрывать происходить представлять . просто подобный положение . последний положение . слово . последний старый комната . принимать странный пока стоять . совершенно замечать . слово . становиться получать . просто подобный интерес . подымать . подруга последний сказать . подумать свой состояние . поставлять конец прибавлять подобный . просто бог принимать сторона . подходить стрелка стоять', 108)
start: деньги взять работа сделать . туда закинуть файл папка забава, predict: ('свой стол довольно сказать . подумать . получать поступать . по

...

start: добавляться подтягивание нагрузка мышца стонать кость гудеть сустав словно мелко, predict: ('подобный подумать . проходить свой странный подниматься . несколько странный весь странный половина некоторый подруга присылать . просто получать стол . терпеть . подымать подобный маленький слово последний весь весь поступать . стоять безопасность . пока подобный страшный . весь пространство подумать . знать откуда продолжать оставаться . судьба находить . последний стол подходить . пора странный поле . подумать постоянно . просто становиться отвечать . подумать . становиться неприятный способ странный подруга спрашивать становиться приходить поступать пока выстрел возвращаться . принимать последний стол скоро подозревать . отвечать . собираться подобный странный объяснять взять . странный слово просто . попадать . весь подруга . весь', 105)
start: аптека недешевый комплекс витамин . соответственно ехать деньги уходить ., predict: ('подумать приниматься кровь . помогать . подозревать . 

start: хороший пытаться возражать спокойно кивнуть . русский делать добавлять продавец, predict: ('малый подобный старый просто воспоминание . подобный подумать . просто отвечать . подходить . весь взгляд показываться волос . подумать просто спрашивать . сказать весь подобный просто воздух старик . открывать соглашаться . подходить просто приходиться сказать подобный . подымать положение . становиться обращаться подходить полагать свой поступать поступать . весь день весь сторона . постоянно подобный старый возможно приходить свой стол . пристально подходить весь несколько старый король сказать . понимать понимать . светлый странный подобный дело . должный . стараться весь подумать просто . последний просто следовать . получать . весь подумать спокойно подходить напротив . понимать подробность весь сторона . принимать просто несколько', 106)
start: поинтересоваться . двести пятьдесят выпаливать продавец вытирать лицо кусок серый, predict: ('высокий волос высокий полностью . подозрение 

start: . успевать опускать рукав прикрывать любопытный глаз местный сброд рука, predict: ('долго спрашивать . просто обращаться просто приставать . сказать вернуться собираться стоять крик . возможность . старый подумать весь получать стол . становиться становиться поставлять свой подозревать уставляться подниматься стол . последний стол . подумать становиться примерно подходить сказать свой старик просто подозревать . смотреть . привычный стол приходиться . подходить просто собираться приводить . выходить . принимать странный показываться состояние просто открывать . становиться получать . становиться просто понимать подруга . подумать . серьезно выходить . наш отношение . подбородок . просто стоять просто отвечать отвечать . мочь подниматься столь называть сторона производить подумать собираться . возвращаться восклицать .', 100)
start: . добрый утро . беспроводной наушник вкладыш . фирма цвет, predict: ('собираться мочь получать . подобный странный подруга . становиться . мочь подро

100%|█████████████████████████████████████████████| 1/1 [00:51<00:00, 51.64s/it]

start: ознакомительный фрагмент . текст предоставлять . прочитывать книга целиком купить, predict: ('весь день сердце . подниматься сторона . приказывать подобный случай принимать положение . просто подобный . становиться свой старый собор . весь мир недоверие . весь случай . просто постоянно подавать политический подумать свой лес . понимать . подобный странный свой просто повертываться весь сторона просто просто подумать становиться давать . весь слово подставлять время . подымать рука подавать . подавать большой голова . посмотреть . проводить . подобный приходить подниматься свой приводить . сказать . подумать . последний приходить . постоянный подумать отвечать . подумать стоять возможность . пристально подходить свой переписывать настоящий собственный стол . поставлять подобный подвиг . старый . признаваться подобный просто', 106)


In [21]:
text.loc[0, 'predicted_text']

'весь возможно самый помещение становиться волна . подобный весь положение . подобный становиться комната подлец . принимать заканчивать голова . направляться сторона посмотреть свой полный стол отвечать . случаться . большой воспитанница . спрашивать . поставлять отвечать . вернуться весь стол . просто подруга . вернуться становиться слабый полковник . поздно высокий поставлять окно дом . вернуться разбираться становиться уставляться . просто обращаться . открывать стол . последний принимать замечать . просто просто подходить заканчиваться . случаться . понимать подарить . весь свой дом . становиться открывать предупреждать . последний . подобный понимать подобный странный конец . несколько мочь продолжать . делать странный пол . становиться весь стараться пока . старый сторона . просто подниматься получать . просто подбородок . помогать толковать . весь невозможно становиться подобный просто становиться получать свой мало показываться . весь старик . постоянно приставать . получать .

In [17]:
df_texts

,preprocessed_text
0,надеяться повезти прохрипеть судорожно сжинать...
1,четыре глухой . четыре глухой владимир федоров...
2,принц . принц война . принц . принц война вера...
3,ведомый огонь . огонь оформление . глава трави...
4,пылкий лорд . пылкий лорд пылкий лорд роман пе...
...,...
10398,земля обетованный . обетованный разрешение изд...
10399,храм хранитель . храм хранитель оформление обл...
10400,война мир . война мир олег дивов война мир див...
10401,ген бессмертие . бессмертие пролог выходить зд...


In [16]:
len(df_bot.loc[0, 'predicted_text'].split())

NameError: name 'df_bot' is not defined

In [103]:
df_bot = generate_text(df_texts[:5])

100%|█████████████████████████████████████████████| 5/5 [02:16<00:00, 27.24s/it]


In [115]:
df_bot.to_csv("df.csv", index=False)

In [104]:
len(df_bot.loc[0, 'predicted_text'].split())

12816

In [105]:
len(df_texts.loc[0, 'preprocessed_text'].split())

12329

In [106]:
len(df_bot.loc[1, 'predicted_text'].split())

1140

In [107]:
len(df_texts.loc[1, 'preprocessed_text'].split())

1058

In [108]:
len(df_bot.loc[2, 'predicted_text'].split())

4785

In [109]:
len(df_texts.loc[2, 'preprocessed_text'].split())

4511

In [126]:
sample_size = df_texts.shape[0] // 20

In [128]:
sample_size

520

In [130]:
df_texts.shape[0] % 20

3

In [131]:
sample_size = df_texts.shape[0] // 20
# df_texts_k = pd.DataFrame(columns=['preprocessed_text'])
i = 0
for j in range(sample_size, df_texts.shape[0], sample_size):
    print(df_texts[i:j])
    i = j
ost = df_texts.shape[0] % 20
print(df_texts[-ost:])

                                     preprocessed_text
0    надеяться повезти прохрипеть судорожно сжинать...
1    четыре глухой . четыре глухой владимир федоров...
2    принц . принц война . принц . принц война вера...
3    ведомый огонь . огонь оформление . глава трави...
4    пылкий лорд . пылкий лорд пылкий лорд роман пе...
..                                                 ...
515  вкус вампир . вкус вампир андрей белянин вкус ...
516  книга . книга милена оформление . глава здравс...
517  медик . медик николай дронт . медик подвал нын...
518  кой сниться сборник . кой сниться сборник снит...
519  сочельник . сочельник максим горький сочельник...

[520 rows x 1 columns]
                                      preprocessed_text
520   второй шанс . книга . второй шанс . книга втор...
521   проходной двор . выезжать семь час утро встава...
522   харуки март часто сниться отель . сон принадле...
523   сон . рассказ отужинать . сидеть молча задумчи...
524   поссориться . поссориться нико

In [134]:
df_texts[-0:]

,preprocessed_text
0,надеяться повезти прохрипеть судорожно сжинать...
1,четыре глухой . четыре глухой владимир федоров...
2,принц . принц война . принц . принц война вера...
3,ведомый огонь . огонь оформление . глава трави...
4,пылкий лорд . пылкий лорд пылкий лорд роман пе...
...,...
10398,земля обетованный . обетованный разрешение изд...
10399,храм хранитель . храм хранитель оформление обл...
10400,война мир . война мир олег дивов война мир див...
10401,ген бессмертие . бессмертие пролог выходить зд...


In [135]:
df_texts

,preprocessed_text
0,надеяться повезти прохрипеть судорожно сжинать...
1,четыре глухой . четыре глухой владимир федоров...
2,принц . принц война . принц . принц война вера...
3,ведомый огонь . огонь оформление . глава трави...
4,пылкий лорд . пылкий лорд пылкий лорд роман пе...
...,...
10398,земля обетованный . обетованный разрешение изд...
10399,храм хранитель . храм хранитель оформление обл...
10400,война мир . война мир олег дивов война мир див...
10401,ген бессмертие . бессмертие пролог выходить зд...


In [132]:
df_texts_sample = pd.DataFrame(columns=['preprocessed_text'])
count = 1
for i in range(df_texts.shape[0]):
    if df_texts_sample.shape[0] == sample_size:
#         df_texts_sample.to_csv(f"{count}_df_texts.csv"i index=False)
        print(df_texts_sample)
        df_texts_sample = pd.DataFrame(columns=['preprocessed_text'])
        count += 1
    df_texts_sample.loc[i, 'preprocessed_text'] = df_texts.loc[i, 'preprocessed_text']
if df_texts_sample.shape[0] > 0:
    df_texts_sample.to_csv(f"{count}_df_texts.csv", index=False)
    print(df_texts_sample)

                                     preprocessed_text
0    надеяться повезти прохрипеть судорожно сжинать...
1    четыре глухой . четыре глухой владимир федоров...
2    принц . принц война . принц . принц война вера...
3    ведомый огонь . огонь оформление . глава трави...
4    пылкий лорд . пылкий лорд пылкий лорд роман пе...
..                                                 ...
515  вкус вампир . вкус вампир андрей белянин вкус ...
516  книга . книга милена оформление . глава здравс...
517  медик . медик николай дронт . медик подвал нын...
518  кой сниться сборник . кой сниться сборник снит...
519  сочельник . сочельник максим горький сочельник...

[520 rows x 1 columns]
                                      preprocessed_text
520   второй шанс . книга . второй шанс . книга втор...
521   проходной двор . выезжать семь час утро встава...
522   харуки март часто сниться отель . сон принадле...
523   сон . рассказ отужинать . сидеть молча задумчи...
524   поссориться . поссориться нико

                                      preprocessed_text
6240  охота бабочка . охота бабочка нора робертс охо...
6241  человек амфибия . часть первый морской дьявол ...
6242  начало путь . начало путь василий . начало пут...
6243  лев николаевич толстой рассказ мопассан май го...
6244  социализм . социализм желать написать ваш книг...
...                                                 ...
6755  империя демон . империя империя демон пролог д...
6756  сказочный повесть перевод . глава первый наруш...
6757  лидия алексеевна ребенок наш класс поступать н...
6758  органический прогресс отношение исторический п...
6759  морской царевна . морской царевна георгий иван...

[520 rows x 1 columns]
                                      preprocessed_text
6760  . амфитеатр старый товарищ память многий год х...
6761  глухой тропа . глухой тропа александр степанов...
6762  сказка изворот . сказка изворот владимир одоев...
6763  общественный брожение русский школа . запомина...
6764  рождение . рождени